# Analytics And Tracking Example

In [ ]:
from adversarial_lab.analytics import *
from adversarial_lab.db import SqlliteDB
from adversarial_lab.core.optimizers import PGD
from adversarial_lab.core.penalties import LpNorm
from adversarial_lab.core.losses import CategoricalCrossEntropy
from adversarial_lab.attacker.whitebox import WhiteBoxMisclassification
from adversarial_lab.core.noise_generators import AdditiveNoiseGenerator
from adversarial_lab.core.preprocessing import PreprocessingFromFunction
from adversarial_lab.core.constraints import POClip, PONoisedSampleBounding

from adversarial_lab.utils import Plotting, PlottingFromDB

from PIL import Image
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

In [ ]:
MODEL = "InceptionV3"       # Supported models: InceptionV3, ResNet50, MobileNetV2

In [ ]:
if MODEL == "InceptionV3":
    from tensorflow.keras.applications import InceptionV3
    from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
    model = InceptionV3(weights='imagenet')
    input_shape = (299, 299, 3)
elif MODEL == "ResNet50":
    from tensorflow.keras.applications import ResNet50
    from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
    model = ResNet50(weights='imagenet')
    input_shape = (224, 224, 3)
elif MODEL == "MobileNetV2":
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    model = MobileNetV2(weights='imagenet')
    input_shape = (224, 224, 3)

In [ ]:
def preprocess(sample, *args, **kwargs):
    input_sample = tf.cast(sample, dtype=tf.float32)
    if len(input_sample.shape) == 2:
        input_sample = tf.expand_dims(input_sample, axis=-1)
        input_sample = tf.image.grayscale_to_rgb(input_sample)

    elif len(input_sample.shape) == 3 and input_sample.shape[-1] == 1:
        input_sample = tf.image.grayscale_to_rgb(input_sample)

    input_tensor = tf.convert_to_tensor(input_sample, dtype=tf.float32)
    resized_image = tf.image.resize(input_tensor, (299, 299))
    batch_image = tf.expand_dims(resized_image, axis=0)
    return preprocess_input(batch_image)

In [ ]:
image = Image.open('data/panda.jpg')
image_array = np.array(image)

predictions = model.predict(preprocess(image_array), verbose=0)
print("Predicted class:", decode_predictions(predictions, top=1)[0][0][1])
print("Predicted class index:", np.argmax(predictions, axis=1)[0])


In [ ]:
loss = CategoricalCrossEntropy(penalties=[LpNorm(p=2, lambda_val=0.001)])
optimizer = PGD(learning_rate=1.0)

constrain1 = POClip(min=-5, max=5)
constrain2 = PONoisedSampleBounding(min=0, max=255)
constraints = [constrain1, constrain2]

noise_generator = AdditiveNoiseGenerator(scale=[0, 255], dist="zeros")
preprocessing = PreprocessingFromFunction.create(preprocess)

In [ ]:
db = SqlliteDB(db_path="whitebox.db")

In [ ]:
analytics = AdversarialAnalytics(
    db=db,
    trackers=[
        ImageTracker(),
        LossTracker(),
        NoiseStatisticsTracker(),
        NoiseTracker(),
        PredictionsTracker()
    ],
    table_name="whitebox",
    force_create_table=True
    
)

In [ ]:
attacker = WhiteBoxMisclassification(
    model=model,
    optimizer=optimizer,
    loss=loss,
    noise_generator=noise_generator,
    constraints=constraints,
    preprocessing=preprocessing,
    analytics=analytics,
    verbose=3)

noise = attacker.attack(image_array, 
                        target_class=924,
                        epochs=50, 
                        on_original=True)

In [ ]:
plotter = PlottingFromDB(db=db, table_name="whitebox")

In [ ]:
plotter.plot_losses()

In [ ]:
plotter.plot_predictions(idx=[388, 924])

In [ ]:
plotter.plot_noise_statistics()